# Import and Config

In [124]:
import pandas as pd
from sqlalchemy import create_engine

Config

In [125]:
username = 'postgres'
password = '1234'
port = 5432
db_name = 'orderplay'
host = 'localhost'

Database Connection

In [126]:
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db_name}')

# Normalization-Database

In [127]:
from sqlalchemy import text

## DDL Create Table

In [128]:
schema_sql = """
CREATE TABLE restaurants (
    restaurant_id BIGINT PRIMARY KEY,
    restaurant_name VARCHAR(255),
    subzone VARCHAR(100),
    city VARCHAR(100)
);

CREATE TABLE orders (
    order_id BIGINT PRIMARY KEY,
    customer_id TEXT ,
    order_placed_at TIMESTAMP NOT NULL,
    order_status VARCHAR(50),
    delivery VARCHAR(50),
    distance DECIMAL(10, 2),
    instructions TEXT,
    cancellation_reason TEXT,
    order_ready_marked TEXT,
    restaurant_id BIGINT REFERENCES restaurants(restaurant_id)
);

CREATE TABLE order_items (
    order_line_id SERIAL PRIMARY KEY,
    order_id BIGINT REFERENCES orders(order_id) ON DELETE CASCADE,
    item_name VARCHAR(255),
    quantity_item INT
);

CREATE TABLE order_finances (
    order_id BIGINT PRIMARY KEY REFERENCES orders(order_id) ON DELETE CASCADE,
    bill_subtotal DECIMAL(10, 2),
    packaging_charges DECIMAL(10, 2),
    discount_construct TEXT,
    restaurant_discount_promo DECIMAL(10, 2),
    restaurant_discout_other DECIMAL(10,2),
    gold_discount DECIMAL(10, 2),
    brand_pack_discount DECIMAL(10, 2)
);

CREATE TABLE order_reviews (
    order_id BIGINT PRIMARY KEY REFERENCES orders(order_id) ON DELETE CASCADE,
    rating DECIMAL(2, 1),
    review TEXT,
    rating_missing VARCHAR(20)
);

CREATE TABLE order_operations (
    order_id BIGINT PRIMARY KEY REFERENCES orders(order_id) ON DELETE CASCADE,
    kpt_duration_m DECIMAL(10,2),
    rider_wait_time DECIMAL(10,2),
    restaurant_compensation TEXT,
    restaurant_penalty TEXT,
    customer_complaint_tag VARCHAR(100),
    kpt_missing VARCHAR(20),
    rider_missing VARCHAR(20)
);
"""

## Execute Create Table

In [120]:
with engine.connect() as conn:
    commands = schema_sql.split(';')

    for q in commands:
        clean_q =  q.strip()
        # print(clean_q)
        if not clean_q:
            continue

        try:

            with conn.begin():
                conn.execute(text(clean_q))
            print(f"create table success : {clean_q}")
    
        except Exception as e:
            error_msg = str(e)
            if "already exists" in error_msg.lower():
                print(f"Error : Duplicate Table({clean_q.splitlines()[0]})")
            else:
                print(f"Error:{error_msg[:100]}")

create table success : CREATE TABLE restaurants (
    restaurant_id BIGINT PRIMARY KEY,
    restaurant_name VARCHAR(255),
    subzone VARCHAR(100),
    city VARCHAR(100)
)
create table success : CREATE TABLE orders (
    order_id BIGINT PRIMARY KEY,
    customer_id TEXT ,
    order_placed_at TIMESTAMP NOT NULL,
    order_status VARCHAR(50),
    delivery VARCHAR(50),
    distance DECIMAL(10, 2),
    instructions TEXT,
    cancellation_reason TEXT,
    order_ready_marked TEXT,
    restaurant_id BIGINT REFERENCES restaurants(restaurant_id)
)
create table success : CREATE TABLE order_items (
    order_line_id SERIAL PRIMARY KEY,
    order_id BIGINT REFERENCES orders(order_id) ON DELETE CASCADE,
    item_name VARCHAR(255),
    quantity_item INT
)
create table success : CREATE TABLE order_finances (
    order_id BIGINT PRIMARY KEY REFERENCES orders(order_id) ON DELETE CASCADE,
    bill_subtotal DECIMAL(10, 2),
    packaging_charges DECIMAL(10, 2),
    discount_construct TEXT,
    restaurant_

## Insert Into Table

In [130]:
# (Data Migration)
with engine.connect() as conn:
    with conn.begin():
        
        conn.execute(text("""
            INSERT INTO restaurants (restaurant_id,restaurant_name,subzone,city)
            SELECT "restaurant_id","restaurant_name","subzone","city"
            FROM stagging_orders
            ON CONFLICT (restaurant_id) DO NOTHING;
                          """))
        
        conn.execute(text("""
            INSERT INTO orders (order_id,customer_id,order_placed_at,order_status,delivery,distance,instructions,cancellation_reason,order_ready_marked,restaurant_id)
            SELECT "order_id","customer_id","order_placed_at", "order_status","delivery","distance","instructions","cancellation___rejection_reason","order_ready_marked","restaurant_id"
            FROM stagging_orders;
                        """))


        conn.execute(text("""
            INSERT INTO order_finances (order_id, bill_subtotal, packaging_charges, discount_construct,restaurant_discount_promo,restaurant_discout_other, gold_discount, brand_pack_discount)
            SELECT  "order_id","bill_subtotal","packaging_charges","discount_construct","restaurant_discount_promo","restaurant_discount_flat_offs_freebies_and_others","gold_discount","brand_pack_discount"
            FROM stagging_orders ; 
                          """))
        
        conn.execute(text("""
            INSERT INTO order_reviews (order_id, rating, review, rating_missing)
            SELECT  "order_id","rating","review","rating_missing"
            FROM stagging_orders ; 
                          """))

        conn.execute(text("""
            INSERT INTO order_operations (order_id, kpt_duration_m,rider_wait_time,restaurant_compensation,restaurant_penalty,customer_complaint_tag,kpt_missing,rider_missing)
            SELECT  "order_id", "kpt_duration_minutes","rider_wait_time_minutes","restaurant_compensation_cancellation","restaurant_penalty_rejection","customer_complaint_tag","kpt_missing","rider_missing"
            FROM stagging_orders ; 
                          """))
        
        conn.execute(text("""
            INSERT INTO order_items (order_id, item_name, quantity_item)
            SELECT 
                "order_id",
                trim(split_part(unnest(string_to_array("items_in_order", ',')), 'x', 2)),
                trim(split_part(unnest(string_to_array("items_in_order", ',')), 'x', 1))::INT
            FROM stagging_orders;
                          """))